# 3D Graphics in JupyterLab using pythreejs
## Part 2: The *threed* module by Yen Lee Loh

*Yen Lee Loh (2023-5-16, 2023-6-8)*

On its own, pythreejs is inconvenient for various reasons:
- pythreejs.TorusBufferGeometry creates a torus whose symmetry axis is the z axis.
- pythreejs.CylinderBufferGeometry creates a frustum whose symmetry axis is the y axis.
- It is not easy to reorient these shapes along desired directions.

The `threed` module includes wrapper functions and utility functions to make 3D graphics easier.  It is demonstrated in the examples below.

Run this Jupyter notebook one cell at a time.  (If you run the whole notebook at once, the 3D graphics may fail to render.)

## Example 1: Draw a dumbbell using `threed.sphere` and `threed.cylinder`

Make sure that `threed.py` and `monospace.png` are in the same directory as this notebook file.  Import the `threed` module:

In [19]:
import threed
import pythreejs as p3j
import importlib; importlib.reload (threed);  # Developer use

Create a list called `objects` and populate it.  Each object is actually a `pythreejs.Mesh`.  As a quick test, combine the objects into a `pythreejs.Scene`:

In [20]:
objects = []
objects.append( threed.sphere([1,1,1], 0.5,color='#99FFFF') )
objects.append( threed.sphere([2,2,2], 0.5,color='#FFFF99') )
objects.append( threed.cylinder([1,1,1], [2,2,2], 0.2, color='#99FF99') )
p3j.Scene(children=[*objects])

Scene(children=(Mesh(geometry=SphereBufferGeometry(heightSegments=24, widthSegments=24), material=MeshPhongMat…

For serious work we need more control, so we will use `threed.render`.  This is actually a wrapper around `pythreejs.Renderer`:

In [21]:
threed.render (objects, imageSize=[640,240])

Renderer(camera=PerspectiveCamera(aspect=2.6666666666666665, children=(DirectionalLight(color='#FFFFFF', posit…

## Example 2: Axis tripod

As before, create a list called `objects`, whose members are instances of `pythreejs.Mesh`, and pass it to `threed.render`:

In [4]:
import numpy as np
objects = []
objects.append ( threed.cylinder ([0,0,0], [4,0,0], radius=.2, color='#FF0000') ) # red x axis
objects.append ( threed.cylinder ([0,0,0], [0,4,0], radius=.2, color='#00FF00') ) # green y axis
objects.append ( threed.cylinder ([0,0,0], [0,0,4], radius=.2, color='#0000FF') ) # blue z axis
objects.append ( threed.sphere ([4,0,0], radius=.6, color='#FF0000') )
objects.append ( threed.sphere ([0,4,0], radius=.6, color='#00FF00') )
objects.append ( threed.sphere ([0,0,4], radius=.6, color='#0000FF') )
threed.render (objects)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

Click and drag to rotate the above graphic.  The camera rotates around the target point (pivot), which in this case is not the origin..   You may prefer to set it explicitly:

In [5]:
threed.render (objects, camTgt=(0,0,0))

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

Use `elev` and `azim` to control the camera's viewpoint.  Below, remember that red, green, and blue tubes indicate x, y, and z directions respectively.

In [6]:
kw = {'camTgt':(0,0,0), 'imageSize':(200,200)}
threed.render (objects, elev='0', azim='0', **kw)   # camera is located at (x,0,0) relative to target

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#FFFFFF', position=(3.0, 5.0, 1.0), quater…

In [7]:
threed.render (objects, '0', '90', **kw)  # camera is located at (0,y,0) relative to target

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#FFFFFF', position=(3.0, 5.0, 1.0), quater…

In [8]:
threed.render (objects, '89', '270', **kw)  # look down along +z axis (don't use 90 degrees!)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#FFFFFF', position=(3.0, 5.0, 1.0), quater…

In [9]:
threed.render (objects, '-89', '270', **kw)  # look up along +z axis (don't use -90 degrees!)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='#FFFFFF', position=(3.0, 5.0, 1.0), quater…

## Example 3: Perspective

In [10]:
objects = []
for i in range(4):
    for j in range(4):
        for k in range(4):
            objects.append ( threed.sphere ([i,j,k], radius=.5, color='#99CCFF') )
threed.render (objects, camFw=[1,1,1], camFov=np.radians(1)) # camera is located in [1,1,1] direction relative to target; narrow FoV causes orthographic projection

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [11]:
threed.render (objects, camFw=[1,1,1], camFov=np.radians(100)) # camera is located in [1,1,1] direction relative to target; wide FoV causes distortion

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

## Example 4: Text

At the moment `threed.billboard` provides some limited functionality for drawing text in a 3D scene, using the bitmap font atlas `monospace.png`.  The following code makes a billboard with the default orientation (rotation), i.e., text lying in the xy plane.  The default viewpoint is not suitable:

In [12]:
objects = []
objects.append ( threed.tripod(length=1,radius=.04) )
objects.append ( threed.billboard ('Hello world', position=[0, 0, 0]) )
threed.render (objects)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

The text is readable from a top-down view:

In [13]:
threed.render (objects, '89.99', '270', camDist=5)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

A `threed.billboard` can be customized using the `rotation` and `fontColor` arguments:

In [14]:
Jld = np.diag([5,4,3])
objects = []
rnd = np.array([[0,0,0],[1,0,0],[0,1,0],[0,0,1],[0,1,1],[1,1,0],[1,0,1],[1,1,1]]) @ Jld # bounding box corners
#======== Add spheres at corners of cuboid
for rd in rnd:
    objects.append ( threed.sphere (rd, radius=.5, color='#9999CC') ) # mark each corner
#======== Add cylinders along edges of cuboid
for u in [0,1]:
    for v in [0,1]:
        for w in [0,1,2]:
            rdA = np.roll ([u,v,0], w) @ Jld
            rdB = np.roll ([u,v,1], w) @ Jld
            objects.append ( threed.cylinder (rdA, rdB, radius=.2, color='#9999CC'
                                             ) )
#======== Label faces of cuboid
objects.append ( threed.billboard ('Top', position=[.5, .5, 1] @ Jld, fontTexture=threed.txFont1, fontColor='#FF0000') )
objects.append ( threed.billboard ('Front', position=[.5, 0, .5] @ Jld, rotation=threed.lookAt([0,-1,0],[0,0,-1]), fontTexture=threed.txFont2, fontColor='#FFFF00') )
threed.render (objects, '30', '270')

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

## Example 5: More examples

In [15]:
import importlib; importlib.reload (threed);  # Developer use

In [16]:
objects = []
points = np.array([[0,0,0],[2,0,0],[0,2,0],[0,0,2],[0,2,2],[2,2,0],[2,0,2],[2,2,2],[1,1,1]])
[objects.append ( threed.sphere (point, radius=np.sqrt(.75), color='#999999') ) for point in points]
objects.append ( threed.billboard ("l'Atomium", position=[1.0, -1.2,0]) )  # add text
objects.append ( threed.billboard ('Fe', position=[2.0, 2.0, 2.9]) )     # add more text
threed.render (objects, '60', '270', zoomOut=1.5)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [17]:
objects = []
objects.append ( threed.billboard ('Red fish', position=[1, -1, 0], fontColor='red') )  # add text
objects.append ( threed.billboard ('Blue fish', position=[1, -2, 0], fontColor='blue') )  # add text
objects.append ( threed.billboard ('Yellow on black', position=[1, -3, 0], fontColor='#FFFF99', fontTexture=threed.txFont2) )  # add text
threed.render (objects, '60', '270', zoomOut=1.5)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [116]:
importlib.reload (threed);  # Developer use

In [117]:
objects = []
objects.append ( threed.tripod(length=1,radius=.04) )
objects.append ( threed.box(1,1,1))
mesh = threed.torus (3, .5, color='#FF9900', tubularSegments=48)
mesh.quaternion = threed.quatUToV ([0,0,1], [1,0,0])   # rotate torus so that its [001] axis actually points along [100]
objects.append ( mesh )
threed.render (objects, camDist=9)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [115]:
objects = []
theta = np.radians(60)
phi   = np.radians(60)
adir  = (np.sin(theta)*np.cos(phi), np.sin(theta)*np.sin(phi), np.cos(theta))
objects.append ( threed.arrow ([0,0,0], adir, radShaft=.03, radHead=.06, fracShaft=.8, radialSegments=48, color='#FFCCFF') )
mesh = threed.torus (1, .02, color='#FF9900', tubularSegments=48); objects.append ( mesh )
mesh = threed.torus (1, .02, color='#999999', tubularSegments=48); mesh.quaternion = threed.quatUToV ([0,0,1], [1,0,0]); objects.append ( mesh )
mesh = threed.torus (1, .02, color='#999999', tubularSegments=48); mesh.quaternion = threed.quatUToV ([0,0,1], [0,1,0]); objects.append ( mesh )
mesh = threed.torus (np.sin(theta), .01, color='#999999', tubularSegments=48);mesh.position=[0,0,np.cos(theta)]; objects.append ( mesh )
threed.render (objects, camDist=3)

Renderer(camera=PerspectiveCamera(aspect=1.3333333333333333, children=(DirectionalLight(color='#FFFFFF', posit…

In [119]:
p3j.CircleGeometry(radius=1, segments=8, thetaStart=0, thetaLength=6.283185307179586)

CircleGeometry()